# PCM for tuning regularization parameters in Ridge regression 


In [4]:
# Import necessary libraries
import PcmPy as pcm
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import exp, sqrt
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [26]:
# Make the training data: 
N = 100 # Number of observations 
Q = 10  # Number of random effects regressors   
P = 10  # Number of variables 
Z = np.random.normal(0,1,(N,Q)) # Make random design matrix 
U = np.random.normal(0,1,(Q,P))*0.5 # Make random effects  
Y = Z @ U + np.random.normal(0,1,(N,P)) # Generate training data  

# Make testing data: 
Zt = np.random.normal(0,1,(N,Q))
Yt = Zt @ U + np.random.normal(0,1,(N,P))

In [30]:
# Build the datasets from the Data and condition vectors 
comp = np.array([0,0,0,0,0,0,0,0,0,0])
M1 = pcm.regression.RidgeDiag(comp, fit_intercept = True)
M1.optimize_regularization(Z,Y)
print('Estimated thetas:', M1.theta_.round(2))
print('Regularisation:', (exp(M1.theta_[1])/exp(M1.theta_[0])).round(2))

Estimated thetas: [-1.43  0.09]
Regularisation: 4.61


In [33]:
# Now you can fit the model 
M1.fit(Z,Y)
Yp = M1.predict(Zt)
R2 = 1- np.sum((Yt-Yp)**2)/np.sum((Yt)**2)
print('r2 :', R2.round(3))

r2 : 0.656


In [25]:
Yp = M1.predict(Zt)

{'ridge__alpha': 10.312258501325761}


TypeError: set_params() takes 1 positional argument but 2 were given

In [ ]:
exp(np.linspace(-1,5,10))

In [ ]:
# Fit the model in to the full group, using a individual scaling parameter for each 
T_gr, theta = pcm.inference.fit_model_group(Y, M, fit_scale=True)


In [ ]:
# crossvalidated likelihood is the same as the group fit for all 
# except the component and noise ceiling model
T_cv, theta_cv = pcm.inference.fit_model_group_crossval(Y, M, fit_scale=True)


In [ ]:
ax = pcm.vis.model_plot(T_cv.likelihood,null_model = 'null',noise_ceiling= 'ceil',upper_ceiling = T_gr.likelihood['ceil'])


In [ ]:
T_gr

In [ ]:
G,_ = M[4].predict(theta[4][:M[4].n_param])
plt.imshow(G)